<a href="https://colab.research.google.com/github/brainopensource/datascience-mvp/blob/ml-analytics-mvp/MVP_Machine_Learning_%26_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP de Machine Learning & Analytics para conclusão da Sprint da Pós-Graduação em Ciência de Dados e Analytics (PUC Rio)

# Aluno Lucas da S. Rocha

## Introdução

Este notebook apresenta o desenvolvimento de um Mínimo Produto Viável (MVP) como parte do segundo módulo da Pós-Graduação em Ciência de Dados e Analytics da PUC-Rio. Utilizando dados abertos recentes sobre a produção de petróleo e gás no Brasil, disponibilizados pela Agência Nacional de Petróleo (ANP).

## Definição do Problema

O petróleo continua sendo um ativo estratégico no cenário energético global, afetado por fatores geopolíticos e pela volatilidade dos mercados internacionais. Nesse cenário, o Brasil se destaca como um dos principais produtores mundiais de petróleo e gás natural, tornando sua produção um tema relevante para análises e tomadas de decisão.

O objetivo deste trabalho é investigar a produção de petróleo e gás natural por poço, utilizando dados que abrangem o período de janeiro de 2020 até o primeiro semestre de 2024. Esse recorte captura as tendências mais recentes da indústria e nos permite explorar os fatores que influenciam a produtividade e a eficiência. A partir da aplicação de técnicas de ciência de dados, buscamos identificar padrões produtivos, anomalias e insights que possam otimizar a alocação de recursos e aumentar a eficiência e previsibilidade operacional e financeira.

A riqueza dos dados por poço oferece oportunidades significativas para análises detalhadas. Através de métodos estatísticos e, futuramente, de algoritmos de aprendizado de máquina, pretendemos explorar diversas aplicações, incluindo:    Análise de Eficiência da Produção, Geração de Curvas de Decaimento de Poços,    Classificação de Poços por Eficiência, Previsão da Produção Total por Campo e   Previsão de Royalties.

Essas análises iniciais geram insights valiosos para a gestão e a operação de poços e campos. O pré-processamento dos dados e a análise exploratória formam a base necessária para futuras implementações de machine learning, facilitando o desenvolvimento de modelos preditivos robustos e a criação de ferramentas analíticas que forneçam uma visão mais profunda das dinâmicas de produção. Isso permitirá, no futuro, decisões mais assertivas e embasadas no contexto da produção de petróleo e gás natural no Brasil, à medida que utilizaremos técnicas de aprendizado supervisionado em nosso estudo.


## Código do MVP

### Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import hashlib
import warnings
import missingno as msno

### Funções auxiliares

  Funções que serão úteis posteriormente no nosso MVP

In [ ]:
# Função para gerar indice unico com hash
def hash_column(row, col1, col2, col3):
    """
    Gera um hash MD5 único para uma linha do DataFrame, combinando os valores
    de duas colunas especificadas.

    Args:
        row (pd.Series): A linha do DataFrame que contém os dados.
        col1 (str): O nome da primeira coluna a ser combinada.
        col2 (str): O nome da segunda coluna a ser combinada.

    Returns:
        str: Um hash MD5 gerado a partir dos valores combinados das duas colunas.
    """
    return hashlib.md5(f"{row[col1]}_{row[col2]}_{row[col3]}".encode()).hexdigest()

### Configuração do Ambiente

Configurações estéticas do pandas para limitar o máximo de linhas, colunas e filtrar avisos.

In [ ]:
# Definindo o limite de exibição colunas de dataframe para None (sem limite)
pd.set_option('display.max_columns', 48)

# Definindo o limite de exibição de linhas de dataframe para 48
pd.set_option('display.max_rows', 48)

# Filter warnings
warnings.filterwarnings('ignore')

### Criando lista de URLs de datasets

Conforme sugerido, os dados foram disponibilizados publicamente no repositório do Github e lidos no formato Raw

[Link do Repositório](https://github.com/brainopensource/datascience-mvp)


In [ ]:
url = "https://raw.githubusercontent.com/brainopensource/datascience-mvp/refs/heads/main/datasets/monthly_production_concat/2015_2024_pre-salt_oil-gas-water_production.csv"
df_production = pd.read_csv(url)
display(df_production)

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]


### Carregamento e concatenação dos datasets

Após possuirmos a lista completa de todas as URLs que referenciam todos os datasets, podemos criar Pandas DataFrames e salvá-los numa lista e depois concatená-los num único dataframe.

### Transformação dos dados

Nessa etapa iremos fazer transformações no nosso dado que viabilizam uma análise mais eficiente e correta. A seguir serão alterados nomes de colunas e seus tipos de dados, será feita a seleção dos atributos que interessam para o nosso MVP e também serão feitas as devidas transformações de unidades de produção de petróleo e gás.

In [ ]:
# Renomeando as colunas para nomes mais simplificados e claros
df_production.columns = [
    "estado", "bacia", "nome_poco_anp", "nome_poco_operador", "campo", "operador",
    "numero_contrato", "periodo", "oleo_bbldia", "condensado_bbldia", "petroleo_bbldia",
    "gas_natural_m3dia_assoc", "gas_natural_m3dia_nassoc", "gas_natural_m3dia_total",
    "volume_gas_royalties_m3mes", "agua_bbldia", "instalacao_destino", "tipo_instalacao",
    "tempo_producao_hspormes", "periodo_carga", "corrente", "grau_api",
    "fracao_destilados_lev_corte", "fracao_destilados_lev_vol", "fracao_destilados_med_corte",
    "fracao_destilados_med_vol", "fracao_destilados_pes_corte", "fracao_destilados_pes_vol",
    "perc_vol_metano", "perc_vol_etano", "perc_vol_propano", "perc_vol_isobutano", "perc_vol_butano",
    "perc_vol_isopentano", "perc_vol_npentano", "perc_vol_hexanos", "perc_vol_heptanos",
    "perc_vol_octanos", "perc_vol_nonanos", "perc_vol_decanos", "perc_vol_undecanos",
    "perc_vol_oxigenio", "perc_vol_nitrogenio", "perc_vol_gas_carbonico", "densidade_glp_gas",
    "densidade_glp_liquido", "pcs_gp_kj_m3", "data_atualizacao"
]

# Criar uma coluna com índice único usando hash
df_production['unique_index'] = df_production.apply(hash_column, axis=1, args=('nome_poco_anp', 'periodo', 'campo'))

# Definir a nova coluna 'indice_unico' como o índice do DataFrame
df_production.set_index('unique_index', inplace=True)

# Exibe as primeiras 5 linhas do dataset concatenado
display(df_production.head(5))

,estado,bacia,nome_poco_anp,nome_poco_operador,campo,operador,numero_contrato,periodo,oleo_bbldia,condensado_bbldia,petroleo_bbldia,gas_natural_m3dia_assoc,gas_natural_m3dia_nassoc,gas_natural_m3dia_total,volume_gas_royalties_m3mes,agua_bbldia,instalacao_destino,tipo_instalacao,tempo_producao_hspormes,periodo_carga,corrente,grau_api,fracao_destilados_lev_corte,fracao_destilados_lev_vol,fracao_destilados_med_corte,fracao_destilados_med_vol,fracao_destilados_pes_corte,fracao_destilados_pes_vol,perc_vol_metano,perc_vol_etano,perc_vol_propano,perc_vol_isobutano,perc_vol_butano,perc_vol_isopentano,perc_vol_npentano,perc_vol_hexanos,perc_vol_heptanos,perc_vol_octanos,perc_vol_nonanos,perc_vol_decanos,perc_vol_undecanos,perc_vol_oxigenio,perc_vol_nitrogenio,perc_vol_gas_carbonico,densidade_glp_gas,densidade_glp_liquido,pcs_gp_kj_m3,data_atualizacao
unique_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
f52ac5c38305be3794a1e3c33b50ad2b,Rio de Janeiro,Campos,7-CRT-49-RJS,7CRT49RJS,BARRACUDA,Petrobras,480000038989755,2015/01,"923,6299",0,"923,6299","14,7007",0,"14,7007","455,15043","3,2824",PETROBRAS 48,UEP,0,NaN,NaN,NaN,< ºC,%,NaN,%,> ºC,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/11/2024
cc08faf9143d9770b43d2397a717f377,Rio de Janeiro,Campos,4-RJS-265A-RJS,4RJS 0265A RJ,TRILHA,Trident Energy,480000037089718,2015/01,"245,5712",0,"245,5712","6,2141",0,"6,2141","192,63727",",3248",PETROBRAS 12,UEP,0,NaN,NaN,NaN,< ºC,%,NaN,%,> ºC,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/11/2024
e55d8086a27e1a06e121e376e2bb3cba,Espírito Santo,Campos,6-BRSA-639-ESS,6BFR1ESS,JUBARTE,Petrobras,480000035609749,2015/01,"18192,7386",0,"18192,7386","603,6013",0,"603,6013","18689,30852","396,7456",FPSO CAPIXABA,UEP,0,NaN,NaN,NaN,< ºC,%,NaN,%,> ºC,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/11/2024
2d024096b105949a88a78ab16c058057,São Paulo,Santos,1-BRSA-594-SPS,1SPS55,SAPINHOÁ,Petrobras,486100038842000,2015/01,"17314,4374",0,"17314,4374","583,5423",0,"583,5423","11011,39524",0,FPSO CIDADE DE SÃO PAULO,UEP,0,NaN,NaN,NaN,< ºC,%,NaN,%,> ºC,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/11/2024
1560cdd99c05d21b8ec52e416d720760,São Paulo,Santos,3-BRSA-788-SPS,3SPS69,SAPINHOÁ,Petrobras,486100038842000,2015/01,"29542,8703",0,"29542,8703","1061,0905",0,"1061,0905","20022,68924",0,FPSO CIDADE DE ILHA BELA,UEP,0,NaN,NaN,NaN,< ºC,%,NaN,%,> ºC,%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/11/2024


In [ ]:
# Separar em dois DataFrames
df_tipo_oleo = df_production.iloc[:, 19:]  # Seleciona as colunas restantes com dados de hidrocarbonetos que não serão usadas nesse trabalho
df_production = df_production.iloc[:, :19]  # Seleciona as primeiras 21 colunas com dados de produção

# Verificar a separação dos datasets, iremos focar nos dados de produção nesse trabalho
display(df_production.head())

,estado,bacia,nome_poco_anp,nome_poco_operador,campo,operador,numero_contrato,periodo,oleo_bbldia,condensado_bbldia,petroleo_bbldia,gas_natural_m3dia_assoc,gas_natural_m3dia_nassoc,gas_natural_m3dia_total,volume_gas_royalties_m3mes,agua_bbldia,instalacao_destino,tipo_instalacao,tempo_producao_hspormes
unique_index,,,,,,,,,,,,,,,,,,,
f52ac5c38305be3794a1e3c33b50ad2b,Rio de Janeiro,Campos,7-CRT-49-RJS,7CRT49RJS,BARRACUDA,Petrobras,480000038989755,2015/01,"923,6299",0,"923,6299","14,7007",0,"14,7007","455,15043","3,2824",PETROBRAS 48,UEP,0
cc08faf9143d9770b43d2397a717f377,Rio de Janeiro,Campos,4-RJS-265A-RJS,4RJS 0265A RJ,TRILHA,Trident Energy,480000037089718,2015/01,"245,5712",0,"245,5712","6,2141",0,"6,2141","192,63727",",3248",PETROBRAS 12,UEP,0
e55d8086a27e1a06e121e376e2bb3cba,Espírito Santo,Campos,6-BRSA-639-ESS,6BFR1ESS,JUBARTE,Petrobras,480000035609749,2015/01,"18192,7386",0,"18192,7386","603,6013",0,"603,6013","18689,30852","396,7456",FPSO CAPIXABA,UEP,0
2d024096b105949a88a78ab16c058057,São Paulo,Santos,1-BRSA-594-SPS,1SPS55,SAPINHOÁ,Petrobras,486100038842000,2015/01,"17314,4374",0,"17314,4374","583,5423",0,"583,5423","11011,39524",0,FPSO CIDADE DE SÃO PAULO,UEP,0
1560cdd99c05d21b8ec52e416d720760,São Paulo,Santos,3-BRSA-788-SPS,3SPS69,SAPINHOÁ,Petrobras,486100038842000,2015/01,"29542,8703",0,"29542,8703","1061,0905",0,"1061,0905","20022,68924",0,FPSO CIDADE DE ILHA BELA,UEP,0
